# Classification on CIFAR

In [ ]:
import time
import json
import copy
from pathlib import Path
import datetime

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision

import ops.trains as trains
import ops.tests as tests
import ops.datasets as datasets
import ops.schedulers as schedulers

In [ ]:
# config_path = "configs/cifar10_general.json"
config_path = "configs/cifar100_general.json"

with open(config_path) as f:
    args = json.load(f)
    
print("args: \n", args)

In [ ]:
dataset_args = copy.deepcopy(args).get("dataset")
train_args = copy.deepcopy(args).get("train")
val_args = copy.deepcopy(args).get("val")

dataset_train, dataset_test = datasets.get_dataset(**dataset_args, root="../data")
dataset_name = dataset_args["name"]
num_classes = len(dataset_train.classes)

dataset_train = DataLoader(dataset_train, 
                           shuffle=True, 
                           num_workers=train_args.get("num_workers", 4), 
                           batch_size=train_args.get("batch_size", 128))
dataset_test = DataLoader(dataset_test, 
                          num_workers=val_args.get("num_workers", 4), 
                          batch_size=val_args.get("batch_size", 128))

print("Train: %s, Test: %s, Classes: %s" % (
    len(dataset_train.dataset), 
    len(dataset_test.dataset), 
    num_classes
))

## Model

### VGG

VGG-19:

In [ ]:
import models.vggnet as vgg

model = vgg.dnn_19(num_classes=num_classes)
# model.load_state_dict(torch.load("models_checkpoints/%s_%s" % (dataset_name, model.name)))
    
xs = torch.randn([3, 3, 32, 32])
ys = model(xs)
print("model: %s, output: %s" % (model.name, ys.size()))

In [ ]:
import models.vggnet as vgg

model = vgg.mcdo_19(num_classes=num_classes)
# model.load_state_dict(torch.load("models_checkpoints/%s_%s" % (dataset_name, model.name)))
    
xs = torch.randn([3, 3, 32, 32])
ys = model(xs)
print("model: %s, output: %s" % (model.name, ys.size()))

In [ ]:
import models.vggnet as vgg
import models.vggnet_dnn_block as vggnet_dnn

model = vgg.VGGNet(vggnet_dnn.BasicBlock, [2, 2, 4, 4, 4],
                   sfilter=(1, 1), num_sblocks=(1, 1, 1, 1, 1),
                   temp=2e1,
                   num_classes=num_classes, name="vgg_dnn_smoothing_19")
# model.load_state_dict(torch.load("models_checkpoints/%s_%s" % (dataset_name, model.name)))
    
xs = torch.randn([3, 3, 32, 32])
ys = model(xs)
print("model: %s, output: %s" % (model.name, ys.size()))

In [ ]:
import models.vggnet as vgg
import models.vggnet_mcdo_block as vggnet_mcdo

model = vgg.VGGNet(vggnet_mcdo.BasicBlock, [2, 2, 4, 4, 4],
                   sfilter=(1, 1), num_sblocks=(1, 1, 1, 1, 1), temp=2e1,
                   num_classes=num_classes, name="vgg_mcdo_smoothing_19")
# model.load_state_dict(torch.load("models_checkpoints/%s_%s" % (dataset_name, model.name)))
    
xs = torch.randn([3, 3, 32, 32])
ys = model(xs)
print("model: %s, output: %s" % (model.name, ys.size()))

## Train

Define a TensorBoard writer:

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "runs/%s_%s/%s" % (dataset_name, model.name, current_time)
writer = SummaryWriter(log_dir)

with open("%s/config.json" % log_dir, "w") as f:
    json.dump(args, f)

print("Create TensorBoard log dir: ", log_dir)

Train the model:

In [ ]:
train_args = copy.deepcopy(args).get("train")
epochs = train_args.pop("epochs")
warmup_epochs = train_args.get("warmup_epochs", 0)
val_args = copy.deepcopy(args).get("val")
n_ff = val_args.pop("n_ff", 1)
gpu = args.get("gpu", False)

optim_args = copy.deepcopy(args).get("optim")
optimizer, train_scheduler = trains.get_optimizer(model, **optim_args)
warmup_scheduler = schedulers.WarmupScheduler(optimizer, len(dataset_train) * warmup_epochs)

if gpu:
    model = model.cuda()

for epoch in range(warmup_epochs):
    batch_time = time.time()
    nll = trains.train_epoch(optimizer, model, dataset_train, warmup_scheduler, gpu=gpu)
    print("The model is warmed up: %.2f sec" % (time.time() - batch_time))

for epoch in range(epochs):
    batch_time = time.time()
    nll = trains.train_epoch(optimizer, model, dataset_train, gpu=gpu)
    train_scheduler.step()
    batch_time = time.time() - batch_time
    
    if (epoch + 1) % 1 == 0:
        writer.add_scalar("train/nll", nll, global_step=epoch)
        template = "(%.2f sec/epoch) Epoch: %d, NLL: %.4f, lr: %.3e"
        print(template % (batch_time,
                          epoch,
                          nll, 
                          [param_group["lr"] for param_group in optimizer.param_groups][0]))
            
    if (epoch + 1) % 1 == 0:
        metrics = tests.test(model, n_ff, dataset_test, num_classes, verbose=False, gpu=gpu)
        
        writer.add_scalar("test/nll", metrics[0], global_step=epoch)
        writer.add_scalar("test/acc", metrics[2][0], global_step=epoch)
        writer.add_scalar("test/acc-90", metrics[2][1], global_step=epoch)
        writer.add_scalar("test/unc-90", metrics[3][1], global_step=epoch)
        writer.add_scalar("test/iou", metrics[4][0], global_step=epoch)
        writer.add_scalar("test/iou-90", metrics[4][1], global_step=epoch)
        writer.add_scalar("test/freq-90", metrics[5][1], global_step=epoch)
        writer.add_scalar("test/top-5", metrics[6], global_step=epoch)
        writer.add_scalar("test/brier", metrics[7], global_step=epoch)
        writer.add_scalar("test/ece", metrics[11], global_step=epoch)

        cal_diag = torchvision.utils.make_grid(metrics[12])
        writer.add_image("test/calibration diagrams", cal_diag, global_step=epoch)

        for name, param in model.named_parameters():
            name = name.split(".")
            writer.add_histogram("%s/%s" % (name[0], ".".join(name[1:])), param, global_step=epoch)


Save the model:

In [ ]:
Path("models_checkpoints").mkdir(parents=True, exist_ok=True)
torch.save(model.state_dict(), "models_checkpoints/%s_%s" % (dataset_name, model.name))

## Test

In [ ]:
gpu = args.get("gpu", False)

if gpu:
    model = model.cuda()

metrics = tests.test(model, 1, dataset_test, num_classes, verbose=True, gpu=gpu)

In [ ]:
gpu = args.get("gpu", False)

if gpu:
    model = model.gpu()

for n_ff in [1, 2, 3, 4, 5, 10, 30, 50]:
    print('N: %s, ' % n_ff, end='')
    metrics = tests.test(model, n_ff, dataset_test, num_classes, verbose=False, gpu=gpu)